# Fine-tuning Lag-Llama

In [2]:
!git clone https://github.com/time-series-foundation-models/lag-llama/

fatal: destination path 'lag-llama' already exists and is not an empty directory.


In [3]:
cd /content/lag-llama

/content/lag-llama


In [4]:
!pip install -r requirements.txt --quiet

We then download our pretrained model weights from [HuggingFace](https://huggingface.co/time-series-foundation-models/Lag-Llama) 🤗

In [5]:
!huggingface-cli download time-series-foundation-models/Lag-Llama lag-llama.ckpt --local-dir /content/lag-llama

/content/lag-llama/lag-llama.ckpt


## Imports

We import the required packages and the lag llama estimator object which we can use to make predictions.

In [6]:
!pip install gluonts==0.14.4

In [7]:
from itertools import islice

from matplotlib import pyplot as plt

from tqdm.autonotebook import tqdm

import torch

from gluonts.evaluation import make_evaluation_predictions
from gluonts.dataset.pandas import PandasDataset

import pandas as pd

from lag_llama.gluon.estimator import LagLlamaEstimator

We create a function for Lag-Llama inference that we can reuse. This function returns the predictions for the given prediction horizon. The forecast will be of shape (num_samples, prediction_length), where `num_samples` is the number of samples sampled from the predicted probability distribution for each timestep.

In [9]:
def get_lag_llama_predictions(dataset, prediction_length, context_length=32, num_samples=20, device="cuda", batch_size=64, nonnegative_pred_samples=True):
    ckpt = torch.load("lag-llama.ckpt", map_location=device)
    estimator_args = ckpt["hyper_parameters"]["model_kwargs"]

    estimator = LagLlamaEstimator(
        ckpt_path="lag-llama.ckpt",
        prediction_length=prediction_length,
        context_length=context_length,

        # estimator args
        input_size=estimator_args["input_size"],
        n_layer=estimator_args["n_layer"],
        n_embd_per_head=estimator_args["n_embd_per_head"],
        n_head=estimator_args["n_head"],
        scaling=estimator_args["scaling"],
        time_feat=estimator_args["time_feat"],

        nonnegative_pred_samples=nonnegative_pred_samples,

        # linear positional encoding scaling
        rope_scaling={
            "type": "linear",
            "factor": max(1.0, (context_length + prediction_length) / estimator_args["context_length"]),
        },

        batch_size=batch_size,
        num_parallel_samples=num_samples,
    )

    lightning_module = estimator.create_lightning_module()
    transformation = estimator.create_transformation()
    predictor = estimator.create_predictor(transformation, lightning_module)

    forecast_it, ts_it = make_evaluation_predictions(
        dataset=dataset,
        predictor=predictor,
        num_samples=num_samples
    )
    forecasts = list(tqdm(forecast_it, total=len(dataset), desc="Forecasting batches"))
    tss = list(tqdm(ts_it, total=len(dataset), desc="Ground truth"))

    return forecasts, tss

In [10]:
device = 'cuda'
prediction_length = 8
context_length = 128
num_samples = 20

ckpt = torch.load("lag-llama.ckpt", map_location=device)
estimator_args = ckpt["hyper_parameters"]["model_kwargs"]

estimator = LagLlamaEstimator(
        ckpt_path="lag-llama.ckpt",
        prediction_length=prediction_length,
        context_length=context_length,

        nonnegative_pred_samples=True,
        aug_prob=0,
        lr=5e-4,

        # estimator args
        input_size=estimator_args["input_size"],
        n_layer=estimator_args["n_layer"],
        n_embd_per_head=estimator_args["n_embd_per_head"],
        n_head=estimator_args["n_head"],
        time_feat=estimator_args["time_feat"],

        batch_size=64,
        num_parallel_samples=num_samples,
        trainer_kwargs = {"max_epochs": 50,}
    )

In [24]:
url = 'https://raw.githubusercontent.com/marcopeix/FoundationModelsForTimeSeriesForecasting/main/data/walmart_sales_small.csv'

df = pd.read_csv(url)
df = df[df['Store'] == 1]

input_df = df[:-8]
test_df = df[-8:]

input_df['Weekly_Sales'] = input_df['Weekly_Sales'].astype('float32')

<ipython-input-24-abc609e88630>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  input_df['Weekly_Sales'] = input_df['Weekly_Sales'].astype('float32')


In [26]:
input_ds = PandasDataset.from_long_dataframe(input_df, target='Weekly_Sales', item_id='Store')

In [27]:
predictor = estimator.train(input_ds, cache_data=True, shuffle_buffer_length=1000)

INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.10/dist-packages/lightning/pytorch/trainer/configuration_validator.py:74: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name          | Type               | Params
-----------------------------------------------------
0 | model         | LagLlamaModel      | 2.4 M 
1 | augmentations | ApplyAugmentat

Training: |          | 0/? [00:00<?, ?it/s]

INFO: Epoch 0, global step 50: 'train_loss' reached 13.64909 (best 13.64909), saving model to '/content/lag-llama/lightning_logs/version_2/checkpoints/epoch=0-step=50.ckpt' as top 1
INFO:lightning.pytorch.utilities.rank_zero:Epoch 0, global step 50: 'train_loss' reached 13.64909 (best 13.64909), saving model to '/content/lag-llama/lightning_logs/version_2/checkpoints/epoch=0-step=50.ckpt' as top 1
INFO: Epoch 1, global step 100: 'train_loss' reached 13.03541 (best 13.03541), saving model to '/content/lag-llama/lightning_logs/version_2/checkpoints/epoch=1-step=100.ckpt' as top 1
INFO:lightning.pytorch.utilities.rank_zero:Epoch 1, global step 100: 'train_loss' reached 13.03541 (best 13.03541), saving model to '/content/lag-llama/lightning_logs/version_2/checkpoints/epoch=1-step=100.ckpt' as top 1
INFO: Epoch 2, global step 150: 'train_loss' reached 12.91437 (best 12.91437), saving model to '/content/lag-llama/lightning_logs/version_2/checkpoints/epoch=2-step=150.ckpt' as top 1
INFO:light

In [28]:
forecast_it, _ = make_evaluation_predictions(
        dataset=input_ds,
        predictor=predictor,
        num_samples=num_samples
    )

In [29]:
forecasts = list(tqdm(forecast_it, total=len(input_ds), desc="Forecasting batches"))

Forecasting batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [30]:
import numpy as np

def get_median_and_ci(data,
                      start_date,
                      horizon,
                      freq,
                      id,
                      confidence=0.95):

    n_samples, n_timesteps = data.shape

    # Calculate the median for each timestep
    medians = np.median(data, axis=0)

    # Calculate the lower and upper percentile for the given confidence interval
    lower_percentile = (1 - confidence) / 2 * 100
    upper_percentile = (1 + confidence) / 2 * 100

    # Calculate the lower and upper bounds for each timestep
    lower_bounds = np.percentile(data, lower_percentile, axis=0)
    upper_bounds = np.percentile(data, upper_percentile, axis=0)

    pred_dates = pd.date_range(start=start_date, periods=horizon, freq=freq)
    formatted_dates = pred_dates.strftime('%m-%d-%Y').tolist()

    # Create a DataFrame with the results
    df = pd.DataFrame({
        'Date': formatted_dates,
        'Store': id,
        'Lag-Llama': medians,
        f'Lag-Llama-lo-{int(confidence*100)}': lower_bounds,
        f'Lag-Llama-hi-{int(confidence*100)}': upper_bounds
    })

    return df

In [31]:
finetuned_preds = get_median_and_ci(forecasts[0].samples,
                                    start_date='2012-09-07',
                                    horizon=8,
                                    freq='W-FRI',
                                    id=1,
                                    confidence=0.80)

finetuned_preds

,Date,Store,Lag-Llama,Lag-Llama-lo-80,Lag-Llama-hi-80
0,09-07-2012,1,1526477.000,1.522845e+06,1.533058e+06
1,09-14-2012,1,1493959.250,1.489004e+06,1.498740e+06
2,09-21-2012,1,1436655.125,1.433330e+06,1.440352e+06
3,09-28-2012,1,1632558.500,1.628580e+06,1.639273e+06
4,10-05-2012,1,1590559.125,1.581929e+06,1.595412e+06
5,10-12-2012,1,1595454.500,1.588975e+06,1.598451e+06
6,10-19-2012,1,1493302.375,1.488031e+06,1.499517e+06
7,10-26-2012,1,1581318.375,1.573675e+06,1.585701e+06


In [32]:
def calculate_mae_smape(pred_df, test_df, target_col, pred_col):

    # Extract the relevant columns
    y_true = test_df[target_col].values
    y_pred = pred_df[pred_col].values

    # Calculate MAE
    mae = int(np.mean(np.abs(y_true - y_pred)))

    # Calculate sMAPE
    denominator = np.abs(y_true) + np.abs(y_pred)
    smape = round(np.mean(2.0 * np.abs(y_true - y_pred) / denominator) * 100,2)

    return mae, smape

In [33]:
mae_finetuned, smape_finetuned = calculate_mae_smape(finetuned_preds, test_df, 'Weekly_Sales', 'Lag-Llama')

print(mae_finetuned, smape_finetuned)

78595 5.07
